## Using the GDAL Python API for Raster Processing

GDAL (Geospatial Data Abstraction Library) is a free, open source library for reading/writing raster and vector geospatial data formats. GDAL tools are chiefly for raster processing - the associated OGR library is used for vector, but these are being folded together. GDAL is used in many applications (including ArcGIS) and has APIs for use in several languages.

The GDAL Python API is well developed and widely used. It can be installed through Anaconda with **conda install -c conda-forge gdal**. Documentation and support isn't great but can be found - one good resource is the [Python GDAL/OGR Cookbook](https://pcjericks.github.io/py-gdalogr-cookbook/raster_layers.html.

In OPP, we utilize this library heavily for doing things like advanced zonal statistics for habitat or watershed overlays. The main advantages of GDAL vs ArcPy or ArcGIS is (1) processing speed, (2) flexibility, and (3) no need for an Esri license.

In this notebook I demonstrate a simple use case for GDAL - manipulating the Cropland Data Layer for a small state (Delaware). I acquired the raster dataset from the [Geospatial Data Gateway](https://datagateway.nrcs.usda.gov/GDGOrder.aspx)

First, let's open a raster file and look at some attributes:

In [ ]:
import os
import gdal
import numpy as np
import pandas as pd

# Path to the raster file
cdl_path = os.path.join("GIS", "nass_de", "cdl_30m_r_de_2016_utm18.tif")

# Open the file
cdl_raster = gdal.Open(cdl_path)

# Get raster geotransform (top_left_x, pixel_width, rotation, top_left_y, rotation, pixel_height)
left, cell_size, _, top, *_ = cdl_raster.GetGeoTransform()  # north up
x_size = cdl_raster.RasterXSize * cell_size
y_size = cdl_raster.RasterYSize * cell_size
right = left + x_size
bottom = top - y_size

print("Cell size: ", cell_size)
print("Left: {}\nRight: {}\nBottom: {}\nTop: {}".format(left, right, bottom, top))

The key to manipulating the raster data is reading it into an array - from there we can do manipulations with NumPy, Pandas, or other Python tools. This code reads a piece of the raster into an array.

In [ ]:
# Extract a 5 km by 5 km piece of the land cover raster as an array
x_offset = 5000  # 10 km east from the western edge
y_offset = 5000  # 10 km south from the northern edge
x_max = 3000  # 3 km wide
y_max = 3000  # 3 km long
band = cdl_raster.GetRasterBand(1)  # 1 band raster
bounds = map(lambda x: int(x / cell_size), (x_offset, y_offset, x_max, y_max))  # meters -> pixels
sample = band.ReadAsArray(*bounds)

print("Shape: {}\n".format(sample.shape))
print(sample)


From here we can use Python tools to analyze or manipulate the data:

In [ ]:
# Return the area for the land cover classes
cdl_table = pd.read_csv(os.path.join("GIS", "cdl_classes.csv")).set_index('cdl')
class_counts = np.array(np.unique(sample, return_counts=True)).T
class_table = pd.DataFrame(class_counts, columns=['cdl', 'pixels']).set_index('cdl')
class_table = class_table.merge(cdl_table, left_index=True, right_index=True)
class_table['area'] = np.int32(class_table.pop('pixels') * cell_size ** 2)
class_table['pct'] = (class_table.area / (x_max * y_max)) * 100

print(class_table.sort_values('pct', ascending=False))

And we can modify the array and convert it back to a raster:

In [ ]:
sample[sample != 141] = 0

# Write the array to a new raster
out_path = os.path.join("GIS", "test_square.tif")
driver = cdl_raster.GetDriver()
out_raster = driver.Create(out_path, int(x_max / cell_size), int(y_max / cell_size), 1, gdal.GDT_Int32)
out_raster.SetGeoTransform((left + x_offset, cell_size, 0, top - y_offset, 0, -cell_size))
out_band = out_raster.GetRasterBand(1)
out_band.WriteArray(sample, 0, 0)

If we rearrange the above code into classes, it makes it easy to do more advanced operations, or to do the same operations repetetively. 

This class initializes a Raster from a path:

In [ ]:
class GDALRaster(object):
    def __init__(self, path):
        self.obj = gdal.Open(path)
        self.left, self.cell_size, _, self.top, *_ = self.obj.GetGeoTransform()
        self.shape = np.array([self.obj.RasterXSize, self.obj.RasterYSize])
        self.x_size, self.y_size = self.shape * self.cell_size


And this one provides all the functionality for making selections and writing to raster like we did above:

In [ ]:
class RasterSelection(object):
    def __init__(self, raster, x_offset=0, y_offset=0, x_max=0, y_max=0):
        if not any((x_offset, y_offset, x_max, y_max)):  # whole raster
            x_max, y_max = raster.x_size, raster.y_size
        self.left = raster.left + x_offset
        self.top = raster.top - y_offset
        self.cell_size = raster.cell_size
        self.x_pixels = int(x_max / raster.cell_size)
        self.y_pixels = int(y_max / raster.cell_size)
        self.bounds = list(map(lambda x: int(x / self.cell_size), (x_offset, y_offset, x_max, y_max)))

        # Get driver from template raster
        self.driver = raster.obj.GetDriver()

        # Fetch the array
        band = raster.obj.GetRasterBand(1)  # 1 band raster
        self.array = band.ReadAsArray(*self.bounds)  # meters -> pixels

    def write(self, out_path):
        out_raster = self.driver.Create(out_path, self.x_pixels, self.y_pixels, 1, gdal.GDT_Int32)
        out_raster.SetGeoTransform((self.left, self.cell_size, 0, self.top, 0, -self.cell_size))
        out_band = out_raster.GetRasterBand(1)
        out_band.SetNoDataValue(0)
        out_band.WriteArray(self.array, 0, 0)
        out_band.FlushCache()

As an example of how to leverage these classes, we'll split up the Delaware raster into tiles.  First, we create a simple function for breaking up a rectangle:

In [ ]:
def make_tiles(x_size, y_size, tile_size):
    x = list(range(0, int(x_size), tile_size)) + [int(x_size)]
    y = list(range(0, int(y_size), tile_size)) + [int(y_size)]
    for i in range(len(x) - 1):
        for j in range(len(y) - 1):
            yield (x[i], y[j], x[i + 1] - x[i], y[j + 1] - y[j])

Now we can iterate through the state and create tiles:

In [ ]:
# Initialize the CDL raster
cdl_raster = GDALRaster(cdl_path)

# Set the output location
out_tile = os.path.join("GIS", "tiles", "tile_{}.tif")

# Initialize a set of tiles to break up the raster
tile_size = 25000
tiles = make_tiles(cdl_raster.x_size, cdl_raster.y_size, tile_size)

for counter, tile in enumerate(tiles):
    print("Processing tile {}...".format(counter))
    sample = RasterSelection(cdl_raster, *tile)
    sample.write(out_tile.format(counter))

And modifying is much simpler:

In [ ]:
# Make a raster of soybean pixels
soy_raster = os.path.join("GIS", "delaware_soy.tif")
sample = RasterSelection(cdl_raster)
sample.array[sample.array != 5] = 0
sample.write(soy_raster)